# Intra-Annual Extremes


In [1]:
%run ../setup.ipynb
%run ../plotting_functions.ipynb
data_dir = Path('../../data' )
output_dir = Path('../../output') 
import sys
sys.path.append(str(Path('..').resolve()))
from maps import get_top_10_table

# We're going to use plotly here, so we need to import it
import plotly.io as pio
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go

# check to make sure that data_dir/rsl_daily_hawaii.nc exists, if not, make warning to run datawrangling notebook
if not (data_dir / 'rsl_hawaii.nc').exists():
    print('rsl_hawaii.nc not found in '+ str(data_dir) +  '. Please run the data wrangling notebook first')
else:
    print('rsl_hawaii.nc found in '+ str(data_dir) +  '. Proceed.')

rsl_hawaii.nc found in ../../data. Proceed.


In [2]:
# open the dataset
# Load dataset with Dask (assuming your dataset is in NetCDF format)
with xr.open_dataset(data_dir / 'rsl_hawaii.nc') as ds:
    sea_level = 0.001*(ds['sea_level'] - ds['MHHW']) #convert to m
    sea_level.attrs['units'] = 'm'
    sea_level.attrs['long_name'] = 'Sea level above MHHW'
    station_names = ds['station_name'].values
    


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [3]:
# resample to daily min, max, and mean
daily_mean = sea_level.resample(time='1D').mean()
daily_max = sea_level.resample(time='1D').max()
daily_min = sea_level.resample(time='1D').min()



# # Group by 'dayofyear' and calculate the mean, max, and min for each day of the year
# avg_daily = daily_mean.groupby('dayofyear').mean(dim='time')
# record_high = sea_level.groupby('dayofyear').max(dim='time')
# record_low = sea_level.groupby('dayofyear').min(dim='time')





In [4]:
# Add a 'dayofyear' coordinate to sea_level only
def add_dayofyear(sea_level):
    return sea_level.assign_coords(dayofyear=sea_level['time'].dt.dayofyear)
daily_mean = add_dayofyear(daily_mean)
daily_max = add_dayofyear(daily_max)
daily_min = add_dayofyear(daily_min)


In [5]:
daily_max

<xarray.DataArray (record_id: 11, time: 43677)> Size: 4MB
array([[  nan,   nan,   nan, ..., 0.167, 0.167, 0.173],
       [  nan,   nan,   nan, ..., 0.018, 0.006, 0.011],
       [  nan,   nan,   nan, ..., 0.084, 0.052, 0.061],
       ...,
       [  nan,   nan,   nan, ..., 0.152, 0.153, 0.149],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ..., 0.253, 0.238, 0.245]])
Coordinates:
  * record_id  (record_id) int64 88B 14 50 52 57 58 59 60 61 547 548 552
  * time       (time) datetime64[ns] 349kB 1905-01-01 1905-01-02 ... 2024-07-31
    dayofyear  (time) int64 349kB 1 2 3 4 5 6 7 ... 207 208 209 210 211 212 213
Attributes:
    units:      m
    long_name:  Sea level above MHHW

In [6]:
avg_daily = daily_mean.sel(time=slice('1983-01-01', '2001-12-31')).groupby('time.dayofyear').mean(dim='time')
avg_high = daily_max.groupby('dayofyear').mean(dim='time')
avg_low = daily_min.groupby('dayofyear').mean(dim='time')

record_low = daily_min.groupby('dayofyear').min(dim='time')
record_high = daily_max.groupby('dayofyear').max(dim='time')


In [7]:
station_names

array(['French Frigate', 'Midway', 'Johnston', 'Honolulu, Hawaii',
       'Nawiliwili', 'Kahului', 'Hilo, Hawaii', 'Mokuoloe',
       'Barbers Point, HI', 'Kaumalapau, HI', 'Kawaihae'], dtype='<U17')

In [36]:
def make_plotlyFigure(mean, max, min, record_high,record_low, rid,ds,station_names):
    figly = go.Figure()
    
    # Select data for the given record_id
    mean_rid = mean.isel(record_id=rid)
    max_rid = max.isel(record_id=rid)
    min_rid = min.isel(record_id=rid)
    record_high_rid = record_high.isel(record_id=rid)
    record_low_rid = record_low.isel(record_id=rid)

    x = mean_rid.dayofyear.values


         # Add traces
    figly.add_trace(go.Scatter(x=x, y=max_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', name='Average High'))
    figly.add_trace(go.Scatter(x=x, y=min_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', fill='tonexty', fillcolor='rgba(0,176,246,0.2)', name='Average High/Low'))
    figly.add_trace(go.Scatter(x=x, y=mean_rid.values, mode='lines', line_color='rgba(0,176,246,1)', name='Average Daily'))
    figly.add_trace(go.Scatter(x=x, y=record_high_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', name='Record High'))
    figly.add_trace(go.Scatter(x=x, y=record_low_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', fill='tonexty', fillcolor='rgba(0,176,255,0.2)', name='Record High/Low'))
    
    # Define tick values (approximate day of the year for the start of each month)
    month_ticks = np.array([1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335])+15
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


    # Update layout
    figly.update_layout(
        title=station_names[rid],
        xaxis_title='Month',
        yaxis_title=ds.attrs['long_name'] + ' [' + ds.attrs['units'] + ']',
        showlegend=True,
        plot_bgcolor='white',
        xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='rgb(200,200,200)', tickvals=month_ticks, ticktext=month_names),
        yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='rgb(200,200,200)', tickformat='.2f'),
        autosize=True,
        legend=dict(orientation='h', y=1.1, x=0.5, xanchor='center', yanchor='top', font=dict(size=10)),
        title_x=0.5,
        title_font_size=24,
        title_font_family='Arial',
        title_font_color='black',
        title_font=dict(size=20, color='black')
    )

    

    return figly

In [39]:
#plot the montly max/mean and min values of Honolulu
figly = make_plotlyFigure(avg_daily, avg_high, avg_low, record_high, record_low, 3,sea_level,station_names)

# add top 10 rankings to this plot
# read ../output/SL_extremes_station_name.csv

CI_dir = Path('../../data/climate_indices' )
with xr.open_dataset(data_dir / 'rsl_hawaii.nc') as ds:
    ds['sea_level'] = 0.001*(ds['sea_level'] - ds['MHHW']) #convert to m
    top_10_table  = get_top_10_table(ds,3,CI_dir)

# add column for dayofyear
top_10_table['dayofyear'] = top_10_table['date'].dt.dayofyear

top_10max = top_10_table[top_10_table['type'] == 'max']
top_10min  = top_10_table[top_10_table['type'] == 'min']
# add top 10 rankings to this plot
figly.add_trace(go.Scatter(x=top_10max['dayofyear'], y=top_10max['sea level (m)'], mode='markers', marker=dict(size=10, color='red'), name='Top 10 Max'))
figly.add_trace(go.Scatter(x=top_10min['dayofyear'], y=top_10min['sea level (m)'], mode='markers', marker=dict(size=10, color='blue'), name='Top 10 Min'))
figly.update_layout(autosize=True,legend=dict(orientation='h', y=1.1, x=0.5, xanchor='center', yanchor='top', font=dict(size=10)),
)
